In [8]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
import json
import logging
import os
from azure.storage.blob import BlobServiceClient
import os
import json
import logging
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

from urllib.parse import urlparse
import urllib.parse


from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

storage_account_connection_string = os.environ["STORAGE_CONNECTION_STRING"]

metadata_folder_name = "sci-curatedmetadata"  #metadata json files storage container
metadata_container_name = "sci-data"  


def load_container_client(storage_account_connection_string, container_name):
    """
    This function establishes a connection with the Azure blob container and returns a container client.

    Parameters:
        account_name (str): The Azure storage account name.
        account_key (str): The Azure storage account key. 
        container_name (str): The name of the blob container in the storage account.
    
    Returns:
        tuple: a tuple containing a status code and the container client. On success, the status code is 1 and on failure, it is 0.
    """
    try:
        #sa_conn_string = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
        blob_service  = BlobServiceClient.from_connection_string(conn_str = storage_account_connection_string)
        container_client = blob_service.get_container_client(container_name)
        logging.info(f"Successfully loaded container client for container {container_name}")
        return 1, container_client
    except Exception as ex:
        logging.error(f"Failed to load container client for container {container_name}. Encountered exception: {ex}")
        return 0, None
    
def get_sci_meta_data(document_url, meta_data_container_client):

    container_name = meta_data_container_client.container_name
    try:
        file_id_folder, extension = parse_document_url(document_url)

        # List blobs in the specified folder
        blob_list = meta_data_container_client.list_blobs(name_starts_with=f"{metadata_folder_name}/{file_id_folder}")

        if blob_list:
            # Iterate through the blobs
            for blob in blob_list:
                # Check if the blob ends with ".json"
                logging.info(blob.name)
                if blob.name.endswith(".json"):
                    # Read the content of the blob and convert it to a dictionary
                    blob_data = meta_data_container_client.download_blob(blob)
                    blob_content = blob_data.readall()
                    json_data = json.loads(blob_content)
                    return dict(json_data)
        else:    
            return None
        # If no JSON file found, return an empty dictionary
    except Exception as e:
        logging.info(e)
        return None
    
def parse_document_url(document_url: str) -> tuple:
    """
    Parse the document URL and extract relevant information.

    Args:
        document_url (str): The URL of the document.

    Returns:
        tuple: A tuple containing container name, category class, report root folder, 
        relative folder path, and file name.
    """
    try:
        #parse url to url split result format
        # EX: SplitResult(scheme='https', netloc='bnlwestgscibd00036.blob.core.windows.net', path='/wor-sharepoint-to-blob-destination-container-dev/CATEGORY_DRINKS_CLASS_1/CW 10 0001/0149/CW100001_CW 14 0001-100070.docx', query='', fragment='')
        parsed_url = urlparse(document_url)
        file_path = parsed_url.path     #reference path
        relative_folder_path, file_name_with_extension = os.path.split(file_path) #seperate relative folder path and file name
        file_name, extension = os.path.splitext(file_name_with_extension) #seperate file name and extension
        file_id_folder = file_name 
        return file_id_folder, extension
    
    except Exception as ex:
        logging.error(f"Failed to fetch relative path for {document_url} ||| Encountered exception: {ex}")
        return "",""

def extract_meta_data(document_url: str, metadata_container: BlobServiceClient):
    """
    Extract metadata from the document URL.

    Args:
        document_url (str): The URL of the document.
        metadata_container (BlobServiceClient): The metadata container
    Returns:
        dict: Metadata extracted from the document URL.
    """
    container = metadata_container.container_name

    if container:
        
        sci_meta_data = get_sci_meta_data(document_url, metadata_container)
        return sci_meta_data
           
    else:
        logging.info("Failed to load metadata container")
        return None
    
def main(body) :

    try:
        records = body["values"]

    except ValueError as e:
        logging.info(e)
        print("Body is not valid JSON.")
    
    try:
        container_connection_status, metadata_container_client = load_container_client(storage_account_connection_string, metadata_container_name)
        if not(container_connection_status):
            print("Failed To Load MetaData Container.")
    except Exception as e:
        logging.info(e)
        print("Failed To Load MetaData Container.")

    try:
        formatted_results = []
        for record in records:
            document_url = record["data"]["metadata_storage_path"]
            print(document_url)
            document_url = urllib.parse.unquote(document_url)
            metadata = extract_meta_data(document_url, metadata_container_client)

            if metadata:
                result ={
                            "curatedMetaData" : metadata
                        }
            else:
                result = {
                            "curatedMetaData": {}
                        }
            logging.info(result)

            formatted_results.append( {
                                "recordId": record["recordId"],
                                "data":  result,
                                "error" : None,
                                "warnings": [{"message": ""} for w in []]
                            })

        print(
            result_to_json(formatted_results)
        )
    except HttpResponseError as e:
        logging.info(e)
        print(
            "Received {0} response from CuratedMetaDataWOR Skill:\n{1}".format(
                e.status_code, e.message
            )
        )
 

def result_to_json(result):
    return json.dumps({"values": result}, default=vars)

In [9]:
body = {
    "values": [
        {
            "recordId": "1",
            "data": {
                "metadata_storage_path": "https://scigenaidata.blob.core.windows.net/sci-data/sci-documents/ARBITRATION PETITION_29_2023.pdf"
            }
        }
    ]
}

main(body)

2024-08-26 10:29:37,039 - INFO - Successfully loaded container client for container sci-data
2024-08-26 10:29:37,043 - INFO - Request URL: 'https://scigenaidata.blob.core.windows.net/sci-data?restype=REDACTED&comp=REDACTED&prefix=REDACTED'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.22.0 Python/3.11.9 (Windows-10-10.0.19045-SP0)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': 'fe442688-6367-11ef-b748-e02be94cfd7d'
    'Authorization': 'REDACTED'
No body was attached to the request


https://scigenaidata.blob.core.windows.net/sci-data/sci-documents/ARBITRATION PETITION_29_2023.pdf


2024-08-26 10:29:39,486 - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/xml'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '8ca3653f-b01e-003e-6674-f7a1fe000000'
    'x-ms-client-request-id': 'fe442688-6367-11ef-b748-e02be94cfd7d'
    'x-ms-version': 'REDACTED'
    'Date': 'Mon, 26 Aug 2024 04:59:39 GMT'
2024-08-26 10:29:39,493 - INFO - sci-curatedmetadata/ARBITRATION PETITION_29_2023
2024-08-26 10:29:39,495 - INFO - sci-curatedmetadata/ARBITRATION PETITION_29_2023/ARBITRATION PETITION_29_2023.json
2024-08-26 10:29:39,500 - INFO - Request URL: 'https://scigenaidata.blob.core.windows.net/sci-data/sci-curatedmetadata/ARBITRATION%20PETITION_29_2023/ARBITRATION%20PETITION_29_2023.json'
Request method: 'GET'
Request headers:
    'x-ms-range': 'REDACTED'
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.22.0 Python/3.11.9 (Wind

{"values": [{"recordId": "1", "data": {"curatedMetaData": {"Case Title": " M/S Arif Azim Co. Ltd. Vs M/S Aptech Ltd.", "Citation": "[2024] 3 S.C.R. 73", "Judges": "D.Y. CHANDRACHUD,J.B. PARDIWALA,Manoj Misra", "Issue for Consideration": "Issue for Consideration Whether the Limitation Act, 1963 is applicable to an application for appointment of arbitrator u/s.11(6), Arbitration and Conciliation Act, 1996; if yes, whether the present petition is barred by limitation; when does the right to whether the court may refuse to make a reference u/s.11 of the Arbitration and Conciliation Act, 1996 where the claims are ex-facie and hopelessly time-barred. Headnotes Arbitration and Conciliation Act, 1996 \u2013 s.11(6) \u2013 Limitation Act, 1963 \u2013 Article 137 \u2013 Applicability", "Headnotes": "Arbitration and Conciliation Act, 1996 &ndash; s.11(6) &ndash; Limitation Act, 1963 &ndash; Article 137 &ndash; Applicability<br>", "Decision Date": " 01-03-2024", "Case Number": " ARBITRATION PETITI